In [1]:
import cv2
import matplotlib.pyplot as plt
import time
import torch
from keras_facenet import FaceNet
from numpy import asarray
import pickle
from facenet_pytorch import MTCNN
import numpy
from PIL import Image


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Preparing

## Evaluation

In [3]:
import numpy as np 
from numpy.linalg import norm
def EvalCosine(vector,data,threshold):
    Name = 'Unknown'
    best = 0
    for obj in data:
        A = vector
        B = obj.vector
        # compute cosine similarity
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        if(cosine>= threshold and cosine >=best):
            Name=obj.name
            best=cosine
    return Name + "_"+str(round( best, 2  ))

## Class to hold the objects

In [4]:
class ProcessedImage:
    def __init__ (self, name , vector):
        self.name = name
        self.vector=vector

## Deserializing feature vector + name

In [5]:
with open('Extracted_Features', 'rb') as f:
    Data = pickle.load(f)

## Declaring agents

In [6]:
embedder = FaceNet()
detector = MTCNN(
    margin=14,
    factor=0.6,
    keep_all=True,
    min_face_size=200,
    device=device) #need to be fine-tuned accordingly to the webcam's resolution


# Deployment

In [7]:
cap = cv2.VideoCapture(0) #change to 0 to switch to the default camera input

if not cap.isOpened():
    print("Cannot open camera")
    exit()

#fps
prev_frame_time = 0
new_frame_time = 0

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 420)  
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 420)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:

        detections,_ = detector.detect(frame)
        #bounding box
        img_crop = []
        if(detections is not None):
            for det in detections:
                cv2.rectangle(frame, (int(det[0]),int(det[1])),(int(det[2]),int(det[3])), (0, 255, 0), 2)
                img_crop.append(frame[int(det[1]):int(det[3]),int(det[0]):int(det[2])])
                #print(det)


        if (len(img_crop) != 0 ):
            #extract features
            new_trainx =list()
                
            for cropped in img_crop:
                embedding = embedder.embeddings(img_crop)
                new_trainx.append(embedding)

            new_trainx = asarray(new_trainx)
            Name=str(EvalCosine(new_trainx[0][0],Data,0.6))
            cv2.putText(frame, Name , (int(det[0]), int(det[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            



        #Display fps
        font = cv2.FONT_HERSHEY_SIMPLEX
        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        fps = str(fps)
        cv2.putText(frame, fps, (7, 70), font, 2, (100, 255, 0), 2, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('Frame', frame)
    
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == 27:
            break
   
    else:
        break
cap.release()
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
